In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import ast

In [3]:
movies = pd.read_csv('/content/drive/My Drive/Machine Learning Projects/Movie Recommendation System/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/My Drive/Machine Learning Projects/Movie Recommendation System/tmdb_5000_credits.csv')

# Preprocessing the data

In [4]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [5]:
credits.head(1)
# credits.head(1)["cast"].values

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
# Joining the two dataframes on Title

In [7]:
movies.merge(credits, on='title').shape

(4809, 23)

In [8]:
movies.shape

(4803, 20)

In [9]:
credits.shape

(4803, 4)

In [10]:
movies = movies.merge(credits, on='title')
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## Remove irrelavant columns
> like budget is mostly irrelevant while genre can be crutial to recommend movies, it is a content based recommendation system, hence numerical data is
 not consided  
> Keep these columns --> [genres, id, keywords, overview, cast, crew]

In [11]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [12]:
# check for missing data
movies.isnull().sum()  # 3 overvies missing remove them

,0
movie_id,0
title,0
overview,3
genres,0
keywords,0
cast,0
crew,0


In [13]:
movies.dropna(inplace=True)

In [14]:
# Check for duplicated data
movies.duplicated().sum()

0

In [15]:
movies.iloc[0].genres  # List of Dictionaries

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

> We want  
  [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]  
  in the following format  
  ['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [16]:
def convert(obj):  # the obj is a list in the form of a string --> unstring it --> ast.literal_eval
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [17]:
movies['genres'] = movies['genres'].apply(convert)

In [18]:
movies.head(2)  # observe the change in genres column

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [19]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [20]:
# I only want the top 3 cast --> first 3 dictionaries

def convert3(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['name'])
      counter += 1
    else:
      break
  return L

In [21]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [22]:
# Same with crew now, but while only considering the director

def fetch_director(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      L.append(i['name'])
      break
  return L

In [23]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [24]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [25]:
# now convert the overview column into a list, which would help us in concatanation

movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


> Make Jhonny Depp --> JhonnyDepp. since there are people with same first names, we do not want Jhonny Depp and Jhonny Sins to be in the same tag. Hence, make it unique.

In [26]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(' ', '') for i in x])

In [27]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


> Now make a tags column combiing everything

In [28]:
movies['tags'] = movies['overview'] + movies['genres'] +movies['cast'] + movies['crew']

In [29]:
new_df = movies[['movie_id', 'title', 'tags']]

In [30]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

<ipython-input-30-ad44e9ca1347>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [31]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [32]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [33]:
# convert to lowercase

new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())
new_df.head(2)

<ipython-input-33-bf12bcf27db0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


# Vectorization

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [35]:
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors.shape

(4806, 5000)

In [36]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
print(list(cv.get_feature_names_out()))

['000', '007', '10', '100', '11', '12', '13', '14', '15', '16', '17', '17th', '18', '18th', '19', '1930s', '1940s', '1944', '1950', '1950s', '1955', '1959', '1960', '1960s', '1962', '1964', '1965', '1967', '1970s', '1971', '1972', '1974', '1976', '1979', '1980', '1980s', '1985', '1999', '19th', '20', '200', '2003', '2009', '20th', '21st', '23', '24', '25', '30', '300', '3d', '40', '50', '500', '60', '60s', '70', '70s', 'aaron', 'aaroneckhart', 'abandoned', 'abducted', 'abigailbreslin', 'abilities', 'ability', 'able', 'aboard', 'abuse', 'abusive', 'academic', 'academy', 'accept', 'accepted', 'accepts', 'access', 'accident', 'accidental', 'accidentally', 'accompanied', 'accomplish', 'accomplished', 'account', 'accountant', 'accused', 'ace', 'achieve', 'act', 'acting', 'action', 'actions', 'activist', 'activities', 'activity', 'actor', 'actors', 'actress', 'acts', 'actual', 'actually', 'adam', 'adammckay', 'adams', 'adamsandler', 'adamshankman', 'adaptation', 'adapted', 'add', 'addict', '

> We can see that we have action, actions which are similar  
> stem --> [loved, loving, love] --> [love, love, love]

In [38]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [39]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [40]:
new_df['tags'] = new_df['tags'].apply(stem)

<ipython-input-40-be18a4346d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [41]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
similarity = cosine_similarity(vectors)  # 4806 x 4806

In [44]:
similarity  # correalation matrix


array([[1.        , 0.15389675, 0.0860663 , ..., 0.        , 0.        ,
        0.        ],
       [0.15389675, 1.        , 0.08830216, ..., 0.03673592, 0.        ,
        0.        ],
       [0.0860663 , 0.08830216, 1.        , ..., 0.03081668, 0.        ,
        0.        ],
       ...,
       [0.        , 0.03673592, 0.03081668, ..., 1.        , 0.07897472,
        0.02746175],
       [0.        , 0.        , 0.        , ..., 0.07897472, 1.        ,
        0.05638839],
       [0.        , 0.        , 0.        , ..., 0.02746175, 0.05638839,
        1.        ]])

# Recommendar System

In [45]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]  # get the index of the movie
  distances = similarity[movie_index]  # find the similarity vecotr of the movie
  movies_list = sorted(list(enumerate(distances)), reverse= True, key=lambda x: x[1])[1:6]  # sort the distances, enumerate so we can maintain the id's while sorting, sort on descending order, take the 2,3,4,5,6 elements
  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [46]:
recommend("Batman Begins")

Amidst the Devil's Wings
The Dark Knight Rises
The Dark Knight
Batman
Dead Man Down


In [47]:
new_df.iloc[1192].title

'Small Soldiers'

# Frontend

In [48]:
import pickle

In [49]:
file_path = '/content/drive/My Drive/Machine Learning Projects/Movie Recommendation System/movies_dict.pkl'

pickle.dump(new_df.to_dict(), open(file_path, 'wb'))

In [50]:
file_path = '/content/drive/My Drive/Machine Learning Projects/Movie Recommendation System/similarity.pkl'

pickle.dump(similarity, open(file_path, 'wb'))